# Reconstruction and combination of multi-coil MR data

This demonstration shows how to reconstruct MR images on a coil-by-coil basis and how to combine the image information from the different receiver coils

This demo is a 'script', i.e. intended to be run step by step in a
Python notebook such as Jupyter. It is organised in 'cells'. Jupyter displays these
cells nicely and allows you to run each cell on its own.


First version: 27th of May 2017  
Updated 26nd of June 2021  
Author: Christoph Kolbitsch, Johannes Mayer  


CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2017 University College London.  
Copyright 2015 - 2017, 2021 Physikalisch-Technische Bundesanstalt.  

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In [ ]:
%matplotlib widget

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('MR', 'c_coil_combination')

## Coil Combination Methods
### Goals of this notebook:
- Explore ways to combine acquisition data from multiple receiver coils.

In [ ]:
__version__ = '0.1.0'

# import engine module
import sirf.Gadgetron as pMR
from sirf.Utilities import examples_data_path
from sirf_exercises import exercises_data_path

# import further modules
import os, numpy
import matplotlib.pyplot as plt

### Coil Combination
#### Goals of this notebook:
- Explore ways to combine acquisition data from multiple receiver coils.

### Multi Receiver Channel Imaging
In principle one does not need multiple coils placed on a patient or phantom in an MR exam. Every scanner has a built-in __body coil__ which is able to receive the MRI signal and one can reconstruct an image from that.

Images from the in-built body coil are rarely used diagnostically. Instead what is used are so-called __receiver coils__, aka "coils", or somethimes refered to as __phased arrays__. When talking about data recorded by different coils these are also referred to as __channels__ (in the literature it always says something of the kind: "_... data were acquired using a 32-channel cardiac coil..._").  

This has several advantages, one being that the receiver can be placed very close to the signal source. Others we will discuss in the following! 

__Important:__ each of this coils also adds a global complex phase onto the recorded signal (hence the name __phased__ arrays!)


#### Naming Convention
The following expressions are usually used synonymously:
- coil sensitivity profile
- coil sensitivity maps (CSM)
- coil maps

In [ ]:
#%% LOAD AND PREPROCESS RAW MR DATA
data_path = exercises_data_path('MR', 'PTB_ACRPhantom_GRAPPA')
filename = os.path.join(data_path, 'ptb_resolutionphantom_fully_ismrmrd.h5')
acq_data = pMR.AcquisitionData(filename)
preprocessed_data = pMR.preprocess_acquisition_data(acq_data)
preprocessed_data.sort()



In [ ]:
#%% RETRIEVE K-SPACE DATA
k_array = preprocessed_data.as_array()
print('Size of k-space %dx%dx%d' % k_array.shape)




In [ ]:
#%% PLOT K-SPACE DATA
k_array = k_array / numpy.max(abs(k_array[:]))
num_channels = k_array.shape[1]

fig = plt.figure()
plt.set_cmap('gray')
for c in range( num_channels ):
    ax = fig.add_subplot(2,num_channels//2,c+1)
    ax.imshow(abs(k_array[:,c,:]), vmin=0, vmax=0.05)
    ax.set_title('Coil '+str(c+1))
    ax.axis('off')
plt.tight_layout()



In [ ]:
#%% APPLY INVERSE FFT TO EACH COIL AND VIEW IMAGES
image_array = numpy.zeros(k_array.shape, numpy.complex128)
for c in range(k_array.shape[1]):
    image_array[:,c,:] = numpy.fft.fftshift(numpy.fft.ifft2(numpy.fft.ifftshift(k_array[:,c,:])))
image_array = image_array/image_array.max()


fig = plt.figure()   
plt.set_cmap('gray')
for c in range(image_array.shape[1]):
    ax = fig.add_subplot(2,num_channels//2,c+1)
    ax.imshow(abs(image_array[:,c,:]), vmin=0, vmax=0.4)
    ax.set_title('Coil '+str(c+1))
    ax.axis('off')
plt.tight_layout()    



### Question:
- What differences appear in the individual channel reconstructions compared to the combined image we saw in the last notebook?


### Sum of Square (SOS) Coil Combination

As you can see the individual receiver channels have a spatially varying intensity due to the coil sensitivity profiles. This information needs to be combined.

When you have a set of independently reconstructed images for each channel $f_c$ where $c \in \{1, \dots N_c \}$ labels the individual coil channels.

One way to combine the signal from all coil channels is to use a sum-of-squares approach:

$$
f_{sos} = \sqrt{ \sum_c \bigl{|} \, f_c \bigr{|}^2 }
$$

In [ ]:
#%% COMBINE COIL IMAGES USING SOS
#image_array_sos = numpy.sqrt(abs(numpy.sum(image_array,1)))
image_array_sos = numpy.sqrt(numpy.sum(numpy.square(numpy.abs(image_array)),1))

image_array_sos = image_array_sos/image_array_sos.max()

fig = plt.figure()
plt.set_cmap('gray')
plt.imshow(image_array_sos, vmin=0, vmax=0.7)
plt.title('Combined image using sum-of-squares') 




### Question:

- Apart from fancy parallel imaging techniques, why is it even useful to have more than one receiver channel?  
- What could be a possible disadvantage of this coil combination approach?
- Why is SOS preferable to simply summing them without squaring: $\;f_{combined} = \sum_c f_c$

### Why Coil Sensitivity Maps?
There are several reasons why one would need to compute the CSMs, e.g.:
1. I want to do parallel imaging (i.e. use spatial encoding provided by CSMs)  
2. I want to improve my SNR.


##### Weighted Sum (WS) Coil Combination
Instead of squaring each channel before adding them up there is another method of combining coils, by weighting each image with it's corresponding CSM.  
If the spatial distribution of the coil sensitivity $C_c$ for each coil $c$ is known then combining images as:  

$$
f_{ws} =  \frac{1}{{\sum_{c'}{\bigl{|} C_{c'} \bigr{|}^2}}}{\sum_c C_c^* \cdot f_c}
$$

yields an optimal signal-to-noise ratio (SNR).    
Note also, that this way of combining channels does not destroy the phase information.

__However,__ for each coil one needs to either
- estimate the coil map $C_c$ from the data itself.
- measure them separately (e.g. in Philips systems).

We will focus on computing them from the data in the following.


### Computing Coil Sensitivities
The blunt approach is to compute the CSMs using the __SRSS__ (Square Root of the Sum of Squares) approach:

$$
    C^{SRSS}_c = \frac{f_c}{f_{sos}}
$$

and to apply some smoothing to the data.

As you can imagine there will be no big SNR difference between $f_{ws}$ and $f_{sos}$ using these coil maps. __We didn't put in any additional effort!__ This works well if the SOS image is homogenous.

__This seems a bit pointless!__ True, combining your images this way will __not give you a gigantic SNR gain__, but you __still get a CSM which you can use for parallel imaging__. And you can generate an coil-combined  image __without losing phase information__ (because we smooth the CSMs!)! 

This all works well when the SOS image is good to begin with. Otherwise, there are more "sophisticated" ways to estimate the coilmaps, e.g. methods named _Walsh_ or _Inati_ which lie beyond of the scope of this workshop.  
__SIRF already provides this functionality__. For the task of estimating coil sensitivities from the acquisition data, in `pMR` there is a class called `CoilSensitivityData`. 


In [ ]:
#%% CALCULATE COIL SENSITIVITIES
csm = pMR.CoilSensitivityData()
#help(csm)

csm.smoothness = 50
csm.calculate(preprocessed_data)

In [ ]:
# Cell plotting the Coilmaps
csm_array = numpy.squeeze(csm.as_array())

# csm_array has orientation [coil, im_x, im_y]
csm_array = csm_array.transpose([1,0,2])

fig = plt.figure()
plt.set_cmap('jet')
for c in range(num_channels):
    ax = fig.add_subplot(2,num_channels//2,c+1)
    ax.imshow(abs(csm_array[:,c,:]))
    ax.set_title('Coil '+str(c+1))
    ax.axis('off')

### Question
Please answer the following questions:
- Why is there noise in some regions of the coilmaps and outside the object? 
- Is this noise in the coilmap going to have a strong negative impact on the image quality in this region of the combined image?
- In which organ in the human anatomy would you expect a coilmap to look similarly noisy?



In [ ]:
#%% COMBINE COIL IMAGES USING WEIGHTED SUM
image_array_ws = numpy.sum(numpy.multiply(image_array, numpy.conj(csm_array)),1)
image_array_ws = abs(numpy.divide(image_array_ws, numpy.sum(numpy.multiply(csm_array, numpy.conj(csm_array)),1)))
image_array_ws = image_array_ws/image_array_ws.max()


diff_img_arr = abs(image_array_sos-image_array_ws)
diff_img_arr = diff_img_arr/diff_img_arr.max()


fig = plt.figure(figsize=[12, 4])
plt.set_cmap('gray')

ax = fig.add_subplot(1,3,1)
ax.imshow(image_array_sos, vmin=0, vmax=0.7)
ax.set_title('Sum-of-squares (SOS)')
ax.axis('off')

ax = fig.add_subplot(1,3,2)
ax.imshow(image_array_ws, vmin=0, vmax=0.7)
ax.set_title('Weighted sum (WS)')
ax.axis('off')

ax = fig.add_subplot(1,3,3)
ax.imshow(diff_img_arr, vmin=-0, vmax=0.1)
ax.set_title('SOS - WS')
ax.axis('off')





### Image Quality Assessment
In low-signal regions you can see from the difference image, a weighted coil combination will give you an improved SNR.  
This dataset was acquired with a head-coil so which is very well-matched, and no flow artifacts appear so the difference is not huge.

### Question: What's "The mysterious next step?":
1. Reconstructing individual channels, add them up and get one image out.
2. Reconstructing individual channels, weight them by their coil maps, add them up and get one image out.
3. Do the mysterious next step, and get one image out. 


### Recap Coil Combination

In this exercise we learned:
- how to combine multichannel image reconstructions
- how to compute a simple coil sensitivity from our data.
- how to employ SIRF to execute this computation for us.


